In [ ]:
!pip install transformers
!pip3 install keybert
from keybert import KeyBERT
kw_model = KeyBERT()
print("keyword model intialized")

from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)

from transformers.pipelines import AggregationStrategy
import numpy as np
import torch
import pandas as pd

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, all_outputs):
        results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy=AggregationStrategy.SIMPLE,
        )
        return np.unique([result.get("word").strip() for result in results])


# Load pipeline
#model_name = "ml6team/keyphrase-extraction-kbir-inspec"
#extractor = KeyphraseExtractionPipeline(model=model_name)

#print("Keyphrase model ready")

# -------------Roberta model --------------

# Use a pipeline as a high-level helper
from transformers import pipeline

#Load pipeline
pipe = pipeline("text-classification", model="facebook/roberta-hate-speech-dynabench-r4-target")

# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

pred_tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
roberta_model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")

print("ROberta model is ready")
print("Both are ready")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.2 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=6ce8db25e69253c17d1c8a6baa8235b53b343459191252ccaf2fc69d74ecd2a5
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2ce9c2da54e713c2cefa19bb8851cf4edad37e5a26939986386575ca1c212e10
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

ROberta model is ready
Both are ready


In [ ]:
import re

def post_process_text_2(text, key_features):



    # Add the prefix to the text
    text = ' $#0@..  ' + text

    # Create a mapping for special characters
    char_mapping = {
        'a': '.ɑ',
        'e': '.ε',
        'i': '1',
        'o': '0.',
        's': '.$',
        'w': '.\/\/',
        'u': '.|_|',
        'n': '.|\|',
          'p' : 'ρ' ,
        'y' : 'ყ',
        'f' :  'ք',
        'v' : 'ѵ',
        'r' : 'г',
        'g' : 'ɡ',
        'l' : 'ɩ',
        'b' : 'ƅ',

    }

    # Replace key features with modified versions
    count = 0
    for feature in key_features:
        if( (count>2 & len(text)<20) | (count>3 & len(text)<30) | (count>4 & len(text)<50) | (count>4 & len(text)>100) ): break

        f = feature.lower()
        sp_count = 0
        skip = False
        modified_feature=""
        for i in range(len(f)) :
          if((sp_count>2 & len(feature)<5) | (sp_count>3 & len(feature)<8) | (sp_count>4 & len(feature)>12) ):
            modified_feature += f[i:]
            break

          c = f[i]
          if(char_mapping.get(c, 0)!=0 and skip == False):
            sp_count +=1
            skip = True
            modified_char = char_mapping.get(c, 0)
            modified_feature += modified_char
          else:
            skip = False
            modified_feature += c

        text = re.sub(r'\b{}\b'.format(re.escape(feature)), modified_feature, text, flags=re.IGNORECASE)

    return text

print("pertubation technique is ready")

pertubation technique is ready


In [ ]:
df = pd.read_csv('/content/Dynamically Generated Hate Dataset v0.2.3.csv')

In [ ]:
'''
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Take the first 10,000 rows after shuffling
#subset_df = df_shuffled.head(10000)
text = []
label = [1 if label_value == 'hate' else 0 for label_value in df_shuffled['label'].values]
for i in range(len(label)):
  if(label[i]): text.append(df_shuffled['text'][i])


print(len(text))
#text = text[:1000]

#print(len(text))
'''

df_test_hate = []

count = 0
for i in range(len(df)):
    if df['split'][i] == 'test':
        count+=1
        df_test_hate.append(df.iloc[i])

# Take the first 10,000 rows after shuffling
#subset_df = df_shuffled.head(10000)
text = []
label = []

for i in range(len(df_test_hate)):
    label_value = df_test_hate[i]['label']
    label.append(label_value)
    if(label[i] =='hate'):
       text.append(df_test_hate[i]['text'])

print(len(text))
print(count)

2268
4120


In [ ]:
"""
Complete workflow pipeline usage
"""

all_kp = []
all_pp = []
pred_id = []
sentiments = []
before_senti = []
p = []
iter = 0

for original_text in text:
    iter += 1

    a = pred_tokenizer(original_text, add_special_tokens=False, return_tensors="pt",padding=True,truncation= True)
    with torch.no_grad():
        logits = roberta_model(**a).logits

    predicted_token_class_ids = logits.argmax(-1)        #debug statement
    p.append(predicted_token_class_ids)

    #print(predicted_token_class_ids)

    predicted_class_id = logits.argmax().item()
    senti = roberta_model.config.id2label[predicted_class_id]
    before_senti.append(senti)

    if (senti == 'hate'):

      keyphrases = [k for (k,a) in kw_model.extract_keywords(original_text)]
      all_kp.append(keyphrases)
      #print(keyphrases)           #debug statement

      perturbed_text = post_process_text_2(original_text, keyphrases)
      all_pp.append(perturbed_text)
      #print(perturbed_text)       #debug statement

      inputs = pred_tokenizer(perturbed_text, add_special_tokens=False, return_tensors="pt",padding=True,truncation= True)

      with torch.no_grad():
          logits = roberta_model(**inputs).logits

      predicted_token_class_ids = logits.argmax(-1)        #debug statement
      pred_id.append(predicted_token_class_ids)
      #print(predicted_token_class_ids)

      predicted_class_id = logits.argmax().item()
      sentiment = roberta_model.config.id2label[predicted_class_id]
      sentiments.append(sentiment)

    else:
      all_kp.append('0')
      all_pp.append('0')
      sentiments.append(senti)

    if(iter%400==0): print(iter)


400
800
1200
1600
2000


In [ ]:
#data = { text, all_pp, all_kp , df['label'].values[:100], sentiment}
data = {
    'Text':text,
    'keyphrases': all_kp,
    'purturbed Text' : all_pp,
    'actua Label' : ['hate' for i in range(len(text))],
    'before Label' : before_senti,
    'after Label' : sentiments
    }
test_hate_ndf = pd.DataFrame(data)
test_hate_ndf['purturbed Text'][1736]

' $#0@..  .|_|nmarried ρe0.pɩe are a ƅuгd.εn to .$oc1etყ'

In [ ]:
test_hate_ndf.to_csv('/content/test_hate_ndf.csv')

In [ ]:
r_count = 0
p_count = 0
for i in range(len(test_hate_ndf['actua Label'])):
  if(test_hate_ndf['actua Label'][i] == test_hate_ndf['before Label'][i]):
    r_count +=1

  if(test_hate_ndf['actua Label'][i] == test_hate_ndf['after Label'][i]): p_count +=1

roberta_accuracy = r_count/len(test_hate_ndf['actua Label'])
perturbed_accuracy = p_count/len(test_hate_ndf['actua Label'])
print(roberta_accuracy)
print(perturbed_accuracy)

0.7367724867724867
0.5348324514991182
